In [1]:
import os
import json
import pandas as pd
import traceback
from dotenv import load_dotenv
import ollama  # Import Ollama

In [2]:
from langchain_ollama.chat_models import ChatOllama

In [3]:
llm = ChatOllama(
    model="llama3.1",
    temperature=0.5
)

In [4]:
llm

ChatOllama(model='llama3.1', temperature=0.5, _client=<ollama._client.Client object at 0x0000026E1E8CB130>, _async_client=<ollama._client.AsyncClient object at 0x0000026E20524D30>)

In [5]:
from langchain_ollama.chat_models import ChatOllama
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [6]:
# Example JSON response template
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}


In [7]:
# Template for generating MCQs using the provided text and guidelines
TEMPLATE = """
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to conform to the text as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guide. \
Ensure to make {number} MCQs.
### RESPONSE_JSON
{response_json}
"""

In [8]:
# PromptTemplate for generating MCQs
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
)

In [9]:
# Chain for generating the quiz
quiz_chain = LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)


C:\Users\shubh\AppData\Local\Temp\ipykernel_28280\23162514.py:2: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  quiz_chain = LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)


In [10]:
# Template for evaluating the quiz generated by the model
TEMPLATE2 = """
You are an expert English grammarian and writer. Given a Multiple Choice Quiz for {subject} students,\
you need to evaluate the complexity of the question and provide a complete analysis of the quiz. Use at max 50 words for complexity analysis. 
If the quiz is not at par with the cognitive and analytical abilities of the students,\
update the quiz questions which need to be changed, and adjust the tone to perfectly fit student abilities.
Quiz_MCQs:
{quiz}

Check from an expert English writer the above quiz:
"""

# PromptTemplate for evaluating and refining the quiz
quiz_evaluation_prompt = PromptTemplate(
    input_variables=["subject", "quiz"],
    template=TEMPLATE2
)

# Chain for evaluating the quiz
review_chain = LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [11]:
# SequentialChain to handle quiz generation and evaluation
generate_evaluate_chain = SequentialChain(
    chains=[quiz_chain, review_chain],
    input_variables=["text", "number", "subject", "tone", "response_json"],
    output_variables=["quiz", "review"],
    verbose=True,
)

In [12]:
# File path to the text file containing the content to generate MCQs
file_path = r"D:\MCQ_Generator\data.txt"

# Read the content from the file
with open(file_path, 'r') as file:
    TEXT = file.read()

print(TEXT)

Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other scientists, bio

In [13]:

# Call the generate_evaluate_chain to generate and evaluate the MCQs
with get_openai_callback() as cb:
    result = generate_evaluate_chain({
        "text": TEXT,
        "number": 5,  # Example: Generate 5 MCQs
        "subject": "Biology",
        "tone": "Normal",
        "response_json": RESPONSE_JSON
    })

C:\Users\shubh\AppData\Local\Temp\ipykernel_28280\2380153764.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  result = generate_evaluate_chain({




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each define

In [14]:
# Access the results from the output dictionary
quiz = result["quiz"]
review = result["review"]

In [15]:
# Output the result
print("Generated Quiz:\n", quiz)  # Generated quiz
print("\nReview and Evaluation:\n", review)  # Evaluation and refinement suggestions

Generated Quiz:
 Here are the 5 multiple choice questions for Biology students:

```
{
    '1': {
        'mcq': 'What is biology defined as?',
        'options': {
            'a': 'The study of non-living things',
            'b': 'The scientific study of life',
            'c': 'The study of the environment',
            'd': 'The study of physics'
        },
        'correct': 'b'
    },
    '2': {
        'mcq': 'What is a major theme in biology that explains the unity and diversity of life?',
        'options': {
            'a': 'Photosynthesis',
            'b': 'Evolution',
            'c': 'Cell division',
            'd': 'Metabolism'
        },
        'correct': 'b'
    },
    '3': {
        'mcq': 'What is one of the ways biologists study life?',
        'options': {
            'a': 'Only at the molecular level',
            'b': 'Only at the population level',
            'c': 'At multiple levels of organization, from molecules to populations',
            'd': 'None of

In [16]:
NUMBER=5 
SUBJECT="Generative AI"
TONE="simple"
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each define

In [17]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:1714
Prompt Tokens:1026
Completion Tokens:688
Total Cost:0.0


In [18]:
quiz=response.get("quiz")

In [22]:
quiz=json.loads(quiz)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [21]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

AttributeError: 'str' object has no attribute 'items'